<a href="https://colab.research.google.com/github/ANNAPAVAN/ML-PROJECTS/blob/main/Sentiment_Analysis(Distil_BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/datasets_files/IMDB_Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df = df.sample(3000)

In [6]:
df["sentiment"].replace({'positive':1,'negative':0},inplace=True)

In [7]:
df.head()

,review,sentiment
16720,Skilled professionals live it up in an exotic ...,0
30183,This film is a good start for novices that hav...,1
118,I have now seen quite a few films by Pedro Alm...,0
37861,OH MY God I am lost Now I know everything this...,0
6081,The contemporary chapter of the U.S. Navy's el...,0


In [8]:
import re
def clean_html(text):
  clean = re.compile('<.*?>')
  return re.sub(clean, '', text)

In [9]:
def convert_lower(text):
  return text.lower()

In [10]:
df["review"]=df["review"].apply(convert_lower)

In [11]:
df.head()

,review,sentiment
16720,skilled professionals live it up in an exotic ...,0
30183,this film is a good start for novices that hav...,1
118,i have now seen quite a few films by pedro alm...,0
37861,oh my god i am lost now i know everything this...,0
6081,the contemporary chapter of the u.s. navy's el...,0


In [12]:
x=df["review"]

In [13]:
y = df["sentiment"]

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [16]:
X_train.shape

(2400,)

In [17]:
X_test.shape

(600,)

In [18]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00


In [19]:

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [20]:
text = "he is pavan , pavan is going to college"
tokens = tokenizer(text)
tokens

{'input_ids': [101, 2002, 2003, 6643, 6212, 1010, 6643, 6212, 2003, 2183, 2000, 2267, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
X_train

18031    my only problem with the curse of the were-rab...
4409     this movie was excellent. it details the strug...
22063    i'm not sure where to start with this. in shor...
10506    is this the same kim ki duk who directed the p...
36230    i had to suffer through this movie three times...
                               ...                        
13983    i was not expecting the powerful filmmaking ex...
2157     man's castle is set in one of those jerry buil...
11799    i will not vote this movie as an awful one, ma...
2643     a group of teens that have broken into a huge ...
5781     i've read just about every major book about th...
Name: review, Length: 2400, dtype: object

In [22]:
X_train = X_train.tolist()

In [23]:
X_test = X_test.tolist()

In [24]:

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)


In [ ]:
train_encodings

In [26]:

y_train

18031    1
4409     1
22063    1
10506    0
36230    0
        ..
13983    1
2157     1
11799    0
2643     1
5781     0
Name: sentiment, Length: 2400, dtype: int64

In [27]:

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [28]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [29]:

from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=100
)


In [30]:

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [31]:

trainer.evaluate(test_dataset)

{'eval_loss': 0.3328722401669151}

In [32]:

trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 1.7555813, -2.0046513],
       [-2.206639 ,  1.9675905],
       [-2.0791306,  1.8933821],
       ...,
       [ 0.9433787, -1.1669314],
       [-1.3956141,  1.2760645],
       [ 2.0948813, -2.44824  ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
    

In [33]:

trainer.predict(test_dataset)[1].shape

(600,)

In [34]:

output=trainer.predict(test_dataset)[1]

In [35]:

from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[317,   0],
       [  0, 283]])

In [36]:

trainer.save_model('senti_model')

In [37]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained("/content/senti_model")


Some layers from the model checkpoint at /content/senti_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/senti_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:


new_review_text = "Movie was ridiculous. I came with high expectations since hero Nikhil has been doing very intriguing films lately but this was a cringe fest. I have a lot of respect for Subhas Chandra Bose ji so was hoping this to be amazing but it looked like they just tried to use his name and story to make it sound like a patriotic movie. It was nothing but an illogical movie with cringy nonsensical jokes at inappropriate timings. Those jokes made the movie shallow and baseless. It’s that kind of typical Telugu movie where the main lead gets away with every trouble even though the bad guys got insane weapons in hand. So superficial."
new_review_encoding = tokenizer(new_review_text, truncation=True, padding=True, return_tensors="tf")


In [39]:


raw_predictions = model(new_review_encoding)[0]

probabilities = tf.nn.softmax(raw_predictions, axis=1)

predicted_label = tf.argmax(probabilities, axis=1).numpy()[0]


In [40]:
predicted_label

0

In [41]:

new_dataset = tf.data.Dataset.from_tensor_slices((dict(new_review_encoding)))


predictions = model.predict(new_dataset)


logits = predictions[0]


sentiment_scores = tf.nn.softmax(logits, axis=1)[:, 1].numpy()

rating_out_of_10 = sentiment_scores * 10

for review, rating in zip(new_review_text, rating_out_of_10):
    print(f"Review: {review}")
    print(f"Rating out of 10: {rating:.2f}")


1/1 [==============================] - 1s 1s/step
Review: M
Rating out of 10: 0.16


In [42]:
review2 = "What a movie it was! The night was absolutely amazing, from the stunning visuals to every aspect being up to the mark. It's safe to say that this is one of Shah Rukh Khan's best movies. Nayanthara delivered an outstanding performance in her scene, even though the film has a Keralite theme. The action sequences and plot twists were exceptionally well-executed, keeping me on the edge of my seat. And let's not forget about the fantastic songs that added another layer of enjoyment to the overall experience. I highly recommend this movie; it's not only a thrilling ride but also suitable for children, making it a perfect family choice. It gave me such an adrenaline rush that I can't stop raving about it!"
review2_encoding = tokenizer(review2, truncation=True, padding=True, return_tensors="tf")

In [43]:
new_dataset = tf.data.Dataset.from_tensor_slices((dict(review2_encoding)))


predictions = model.predict(new_dataset)


logits = predictions[0]


sentiment_scores = tf.nn.softmax(logits, axis=1)[:, 1].numpy()

rating_out_of_10 = sentiment_scores * 10

for review, rating in zip(review2, rating_out_of_10):
    print(f"Review: {review}")
    print(f"Rating out of 10: {rating:.2f}")

1/1 [==============================] - 4s 4s/step
Review: W
Rating out of 10: 9.89
